In [ ]:
import random #usar para a geração de numeros aleatorios
import math #biblioteca matemática padrão do python


# o arquivo é composto por quantidade de objetos, capacidade maxima de cada caixas, e o peso dos objetos
# nesse caso é ignorado a quantidade de objetos, a capacidade é atribuida na variável capMAX e o peso na variavel objeto
nome_arquivo = 'objetos.txt'
with open(nome_arquivo, 'r') as arquivo:
    linhas = arquivo.readlines() #lê todas as linhas do arquivo


capMAX = int(linhas[1].strip()) #Pegando a capacidade maxima de cada caixa
objetos = [int(linha.strip()) for linha in linhas[2:]]

caixas = [] #vetor dinamico, inciando com uma caixa vazia
cap_caixas = [] #capacidade de cada caixa individual 


caixas.append([]) #criando uma caixa vazia
cap_caixas.append(capMAX) #tamanho da primeira caixa 

objetos.sort(reverse = True) #ordena os pesos em ordem decrescente O(n log n)

#First Fit Decreasing
for obj in objetos: 
    inserido = 0 #sempre volta para false 
    for i in range(len(caixas)): 
        if obj <= cap_caixas[i]: #se o objeto cabe na caixa
            cap_caixas[i] -= obj #atualiza o tamanho da caixa (i)
            caixas[i].append(obj)  #insere o objeto na caixa (i)
            inserido = 1 #apenas para saber teve alguma caixa que coube o item (k)
            break #significa que o objeto foi posto em uma caixa que ja existia

    if inserido == 0: # significa que o pacote não coube em nenhuma das caixas existentes
        caixas.append([obj]) # cria mais uma caixa e insere o elemento la dentro
        cap_caixas.append(capMAX - obj) # cria e atualiza a capacidade da nova caixa criada


def gerar_vizinhos(caixas, cap_caixas):
    novas_caixas = [caixa[:] for caixa in caixas] #cria uma copia idependente das caixas
    novas_cap_caixas = cap_caixas[:]
    #escolhe uma caixa aleatoria que tenha pelo menos um objeto
    posi_caixa = random.choice([i for i in range(len(novas_caixas)) if len(novas_caixas[i]) > 0]) 

    #escolhe aleatoriamente um item na caixa posi_caixa
    item = random.choice(novas_caixas[posi_caixa])

    for i in range(len(novas_caixas)):
        if i != posi_caixa and novas_cap_caixas[i] >= item:
            novas_caixas[posi_caixa].remove(item)
            novas_cap_caixas[posi_caixa] += item 

            novas_caixas[i].append(item)
            novas_cap_caixas[i] -= item

            #se conseguir gerar uma vizinhança retorna a solução
            return novas_caixas,novas_cap_caixas
        
    #se não conseguir gerar uma vizinhança, retorna a mesma solução de entrada
    return novas_caixas, novas_cap_caixas


def f(caixas): #proporção de preenchimento
    total_proporcao = 0
    
    for caixa in caixas:
        total_proporcao += sum(caixa) / capMAX
    
    return total_proporcao / len(caixas)


melhor_solucao = caixas #melhor solução obitida ate agora 
iterT = 0 #numero de iterações
T = 1000 #temperatura corrente
sMAX = 500 #numero maximo de iterações

while(T > 0.1): #enquanto a temperatura for maior que 0 (recozimento)
    while(iterT < sMAX): #numero maximo de iterações
        iterT += 1 #sempre aumenta a iteração
        novas_caixas, novas_cap_caixas = gerar_vizinhos(caixas, cap_caixas) #geração de vizinhos

        if len(novas_caixas) < len(caixas): #se a quantidade de caixas da vizinhança for menor, significa que foi utilizado menos caixas
            delta = -1 #delta = -1 para poder entrar no if
        else:
            f_novas_caixas = f(novas_caixas) #soma total da novas_caixas
            f_caixas = f(caixas) #soma total das caixas
            delta =  f_novas_caixas - f_caixas #subtrai o peso total das duas respostas

        if(delta < 0): #significa que a vizinhança é melhor do que a resposta de agora
            caixas = novas_caixas #logo a melhor solução passa a ser a vizinhança
            if(f_novas_caixas < f(melhor_solucao)): #se a vizinhança for menor que a melhor solução global
                melhor_solucao = novas_caixas #atualiza a melhor solução global
        else:
            x = random.uniform(0, 1) #gera um valor aleatorio de ponto flutuante dentre o range 0 e 1
            if x < math.exp(-delta/T):
                caixas = novas_caixas#aceita a nova solução com base na probabilidade, mesmo que essa solução seja pior que a outra
    T = 0.9 * T
    iterT = 0   
caixas = melhor_solucao
cont = 1

caixas = [caixa for caixa in caixas if len(caixa) > 0] #se tiver alguma caixa vazia no final, tem que retirar todas

print("quantidade de caixas",len(caixas))

for i in range(len(caixas)):
    print("caixa[", i+1, "] ", caixas[i])